## model 3: RF+FS4(borda agg)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../models/rf_df_test_fsborda_pred.csv',sep=';', index_col=0)

In [3]:
df.head()

,var38,saldo_var5,var3,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_hace2,saldo_medio_var5_ult3,imp_op_var40_efect_ult1,imp_op_var39_efect_ult3,imp_op_var41_efect_ult3,...,num_reemb_var33_ult1,imp_sal_var16_ult1,imp_trasp_var17_in_hace3,imp_venta_var44_hace3,num_var29_0,num_var43_recib_ult1,num_var6_0,TARGET,prob_0,prob_1
50595,77930.250000,781.47,2,42.03,567.72,553.11,387.63,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,9,0,0,0.874164,0.125836
5252,165125.040000,3.00,2,54195.09,3.00,3.00,18067.02,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0.860393,0.139607
21956,82636.440000,60.00,2,45.51,60.00,60.00,55.17,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0.829680,0.170320
42714,67460.550000,293.70,2,772.29,678.57,1464.06,971.64,0.0,780.0,780.0,...,0,0.0,0.0,0.0,0,0,0,0,0.660396,0.339604
18494,117310.979016,3.00,2,1.86,3.00,3.00,2.61,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,0,0,0,0.891108,0.108892


prob_0	= probabilidade de estar satisfeito  
prob_1 = probabilidade de churn

In [4]:
def ks(data=None,target=None, prob=None):
    ## finding at: https://www.listendata.com/2019/07/KS-Statistics-Python.html
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 5)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable.index = range(1,6)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    # print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [5]:
from scipy.stats import ks_2samp
yhat = df['prob_1']
y = df['TARGET']
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic
ks_stat(y, yhat)

0.4807880114383368

In [6]:
results_quintil = ks(df, target='TARGET', prob='prob_1')
results_quintil

KS is 47.9% at decile 1


,min_prob,max_prob,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS
Decile,,,,,,,,,
1,0.410579,0.872330,400,2628,66.0,18.0,66.0,18.0,47.9
2,0.254734,0.410455,94,2960,15.0,20.0,81.0,38.0,43.1
3,0.134750,0.254727,64,2976,11.0,20.0,92.0,59.0,33.3
4,0.079343,0.134724,35,3003,5.8,21.0,98.0,79.0,18.5
5,0.033459,0.079332,14,3030,2.3,21.0,100.0,100.0,0.0


In [7]:
from sklearn.metrics import roc_auc_score

print("Roc Auc: ",roc_auc_score(df['TARGET'], df['prob_1']))
print("Gini: ",2*(roc_auc_score(df['TARGET'], df['prob_1']))-1)

Roc Auc:  0.8021809225090709
Gini:  0.6043618450181418


In [8]:
results_quintil.columns = ['min_prob','max_prob','Qtd de eventos (Target)','Qtd de não-eventos (Target)','% de eventos no Decil','% de não-eventos no Decil','% de evento acumulados','% de não-evento acumulados','KS']
results_quintil.to_csv("results_model3_rf_fs4.csv",sep=';')